# Exploring the Oura API

In [ ]:
from datetime import date, datetime, timedelta
import os

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool
from bokeh.io import output_notebook
import pandas as pd
import pprint
import requests
from dotenv import load_dotenv

In [ ]:
output_notebook()
load_dotenv()

## Helper class

In [ ]:
class OuraAPIClient:

    ENDPOINT_TO_API_VERSION = {
        "activity": "v1",
        "bedtime": "v1",
        "daily_activity": "v2",
        "heartrate": "v2",
        "personal_info": "v2",
        "readiness": "v1",
        "session": "v2",
        "sleep": "v1",
        "tag": "v2",
        "workout": "v2",
    }

    API_VERSION_TO_BASE_URL = {
        "v1": "https://api.ouraring.com/v1",
        "v2": "https://api.ouraring.com/v2/usercollection",
    }

    API_VERSION_TO_DATE_POSTFIX = {"v1": "", "v2": "_date"}

    def __init__(self, personal_token):
        self.personal_token = personal_token

    def __call__(self, endpoint: str, start: str = None, end: str = None):
        api_version = self.ENDPOINT_TO_API_VERSION[endpoint]
        base_url = self.API_VERSION_TO_BASE_URL[api_version]
        start_param = f"start{self.API_VERSION_TO_DATE_POSTFIX[api_version]}"
        end_param = f"end{self.API_VERSION_TO_DATE_POSTFIX[api_version]}"
        url = f"{base_url}/{endpoint}"  # TODO more robust URL joining
        if start is None:
            params = None if end is None else {end_param: end}
        else:
            params = {
                start_param: start,
                end_param: end,
            }  # TODO what if end is None? test this
        headers = {"Authorization": f"Bearer {self.personal_token}"}
        response = requests.request("GET", url, headers=headers, params=params)
        return response.json()  # TODO check status code and handle errors


client = OuraAPIClient(os.environ["OURA_PERSONAL_ACCESS_TOKEN"])

## v2 API

### Daily Activity

In [ ]:
da = client("daily_activity", "2022-06-25", "2022-06-26")

In [ ]:
pprint.pprint(da["data"][0])

In [ ]:
sorted(da["data"][0]["contributors"].keys())

### Heart Rate

In [ ]:
e = datetime.now()
s = e - timedelta(hours=12)

In [ ]:
hr = client("heartrate", s.isoformat(), e.isoformat())
hr

### Personal Info

In [ ]:
pi = client("personal_info")
pi

### Sessions

In [ ]:
sessions = client("session", "2022-06-01", "2022-06-26")
sessions

### Tags

In [ ]:
tags = client("tag", "2022-06-01", "2022-06-26")
tags

### Workouts 

In [ ]:
workouts = client("workout", "2022-06-01", "2022-06-26")
workouts

## v1 API

### Sleep

In [ ]:
sleep = client("sleep", "2022-06-01", "2022-06-26")
sleep

In [ ]:
dates = []
hr_average = []
hr_lowest = []
for day in sleep["sleep"]:
    dates.append(day["summary_date"])
    hr_average.append(day["hr_average"])
    hr_lowest.append(day["hr_lowest"])
hr_df = pd.DataFrame(
    {"Date": pd.to_datetime(dates), "HR_average": hr_average, "HR_lowest": hr_lowest}
)
hr_df

In [ ]:
hr_cds = ColumnDataSource(hr_df)
p = figure(title="Heart Rate")
p.line(
    x="Date",
    y="HR_average",
    legend_label="HR average",
    line_color="gold",
    line_width=2,
    source=hr_cds,
)
p.line(
    x="Date",
    y="HR_lowest",
    legend_label="HR lowest",
    line_color="indigo",
    line_width=2,
    source=hr_cds,
)
p.legend.location = "top_right"
p.add_tools(CrosshairTool())
tooltips = [
    ("Date", "@Date{%F}"),
    ("HR average", "@{HR_average}"),
    ("HR lowest", "@{HR_lowest}"),
]
p.add_tools(HoverTool(tooltips=tooltips, formatters={"@Date": "datetime"}))
p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)

### Activity Summaries

In [ ]:
activity = client("activity", "2022-06-01", "2022-06-26")
activity

### Readiness Summaries

In [ ]:
readiness = client("readiness", "2022-06-01", "2022-06-26")
readiness

### Ideal bedtime

In [ ]:
bedtime = client("bedtime", "2022-06-01", "2022-06-26")
bedtime